In [3]:
import numpy as np
import torch
from torch import nn
import math
import matplotlib.pyplot as plt

In [4]:
# Carregar os dados para treinar a GAN
dadosPreparados = np.load('valores_pacotes_normalizados.npy')
print(dadosPreparados)

[[0.33333333 0.41666667 0.         ... 0.41666667 0.66666667 0.        ]
 [0.26666667 0.33333333 0.         ... 0.33333333 0.53333333 0.        ]
 [0.33333333 0.41666667 0.         ... 0.41666667 0.66666667 0.        ]
 ...
 [0.33333333 0.41666667 0.         ... 0.41666667 0.66666667 0.        ]
 [0.26666667 0.33333333 0.         ... 0.33333333 0.53333333 0.        ]
 [0.33333333 0.41666667 0.         ... 0.41666667 0.66666667 0.        ]]


In [8]:
print(len(dadosPreparados[0]))

52


In [5]:
torch.manual_seed(111)

## Discriminator


In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(52, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [10]:
discriminator = Discriminator()

## Gerador

In [31]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(10, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 52),
        )

    def forward(self, x):
        output = self.model(x)
        return output
    
generator = Generator()

## Treino

In [33]:
taxaAprendizado = 0.001
numeroEpocas = 300
funcaoPerda = nn.BCELoss()

In [34]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=taxaAprendizado)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=taxaAprendizado)

In [35]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    dadosPreparados, batch_size=batch_size, shuffle=True
)

In [36]:
for epoch in range(numeroEpocas):
    for n, real_samples in enumerate(train_loader):
        # Data para treinamento do discriminador
        real_samples_labels = torch.ones((batch_size, 1))
        latent_space_samples = torch.randn((batch_size, 10))
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1))
        print(generated_samples)
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Treinamento do discriminador
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = funcaoPerda(output_discriminator, all_samples_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data para treinamento do gerador
        latent_space_samples = torch.randn((batch_size, 52))

        # Treinamento do gerador
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = funcaoPerda(
            output_discriminator_generated, real_samples_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

tensor([[ 0.1449, -0.0281,  0.0513,  ..., -0.0731, -0.2205, -0.0264],
        [ 0.2751, -0.0769,  0.0643,  ..., -0.1315, -0.1913, -0.0309],
        [ 0.3226, -0.0444,  0.1299,  ...,  0.0483, -0.1423,  0.0506],
        ...,
        [ 0.1396, -0.0566,  0.0430,  ..., -0.1196, -0.1712, -0.0478],
        [ 0.1562, -0.1230, -0.0035,  ..., -0.0983, -0.2685, -0.0212],
        [ 0.0667, -0.1480, -0.0714,  ..., -0.2454, -0.2454, -0.0715]],
       grad_fn=<AddmmBackward0>)


RuntimeError: mat1 and mat2 must have the same dtype